# Sissejuhatus promptide kujundamisse
Promptide kujundamine on protsess, mille käigus luuakse ja optimeeritakse juhiseid loomuliku keele töötlemise ülesannete jaoks. See hõlmab sobivate juhiste valimist, nende parameetrite häälestamist ja tulemuste hindamist. Promptide kujundamine on oluline, et saavutada NLP mudelite kõrge täpsus ja efektiivsus. Selles jaotises uurime promptide kujundamise põhialuseid, kasutades OpenAI mudeleid.


### Harjutus 1: Tokeniseerimine
Uuri tokeniseerimist, kasutades tiktokenit, OpenAI avatud lähtekoodiga kiiret tokeniseerijat.  
Vaata [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) rohkemate näidete jaoks.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Harjutus 2: Kontrolli Github Models võtme seadistust

Käivita allolev kood, et veenduda, et sinu Github Models lõpp-punkt on õigesti seadistatud. Kood proovib lihtsat põhiküsimust ja valideerib vastuse. Sisend `oh say can you see` peaks täienduma umbes järgmiselt: `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Harjutus 3: Väljamõeldised
Uurige, mis juhtub, kui palute LLM-il anda vastuseid teemal, mis võib olla olematu, või teemadel, millest see ei pruugi teada, kuna need jäävad selle eelnevalt treenitud andmekogumi (uuemad) piiridest välja. Vaadake, kuidas vastus muutub, kui proovite teistsugust sisendit või teistsugust mudelit.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Harjutus 4: Juhiste põhjal
Kasuta muutujat "text", et määrata põhisisu,  
ja muutujat "prompt", et anda juhis, mis on seotud selle põhisisuga.

Siin palume mudelil kokku võtta teksti teise klassi õpilasele.


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Harjutus 5: Keeruline päring  
Proovi päringut, mis sisaldab süsteemi, kasutaja ja assistendi sõnumeid  
Süsteem määrab assistendi konteksti  
Kasutaja ja assistendi sõnumid loovad mitme pöördega vestluse konteksti  

Pane tähele, kuidas assistendi isiksus on süsteemi kontekstis määratud "sarkastiliseks".  
Proovi kasutada teistsugust isiksuse konteksti. Või katseta teistsugust sisend/väljund sõnumite seeriat.


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Harjutus: Uuri oma intuitsiooni
Ülaltoodud näited annavad sulle mustrid, mida saad kasutada uute juhiste loomiseks (lihtsad, keerulised, juhised jne) - proovi luua teisi harjutusi, et uurida mõningaid teisi ideid, millest oleme rääkinud, nagu näiteks näited, vihjed ja palju muud.



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
